# Build RAPTOR RAG

*_Algorithm_*
- Input: array of strings
- Step 1: split each string into contiguous chunks with len <= 100
- Step 2: group chunks into clusters
- step 3: summarize the clusters

Ref: https://github.com/langchain-ai/langchain/blob/master/cookbook/RAPTOR.ipynb

## Lessons
- split the orginal text into chunks
- embed question into document

In [5]:
import sys
sys.path.append("../..")
from src.utils.text_utils import create_splitter

splitter = create_splitter(chunk_size=400, overlap=50, separators=[".", "\n"])

## Load data

In [6]:
from datasets import load_dataset
# REPO = 'BroDeadlines/TEST.edu_tdt_proposition_data'
# SPLIT = 'TEST.basic_index_TDT_clean'
REPO = "BroDeadlines/TEST.UEH.ueh_copora_data"
SPLIT = 'train'
repo_data = load_dataset(REPO, split=SPLIT)
repo_data

Dataset({
    features: ['url', 'content', 'doc_id', 'metadata', 'split', 'shards', 'propositions', 'proposition_list'],
    num_rows: 257
})

In [7]:
from src.utils.type_utils import create_langchain_doc

docs_texts = []

def make_langchain_documents2(row):
    docs = splitter.create_documents([row['content']], [{"doc_id": row['doc_id']}])
    docs_texts.extend(docs)
    return row
    
d = repo_data.map(make_langchain_documents2)
len(docs_texts)

0

In [4]:
docs_texts[0].metadata['doc_id'] == docs_texts[1].metadata['doc_id']

True

In [6]:
len(docs_texts)

770

## Build RAPTOR

In [7]:
from src.rag.raptor import RAPTOR
from src.service.provider import ProviderService
provider = ProviderService()
raptor = RAPTOR(provider)

vec_idx = "vec-raptor-ueh-tree"
text_idx = "text-raptor-ueh-tree"

es = provider.get_elasticsearch_store(vec_idx)
bm = provider.get_elasticsearch_bm25(text_idx)

In [8]:
leaf_ids = [d.metadata['doc_id'] for d in docs_texts]
leaf_texts = [d.page_content for d in docs_texts]
results = raptor.recursive_embed_cluster_summarize(leaf_texts, leaf_ids, level=1, n_levels=3)

raptor.log - ERROR - [ERROR] .
Phấn đấu học tập thật tốt từ khi mới vào trường để đạt được mục tiêu.
Quản lý giữa công việc và việc học tập một cách hợp lý
1. ĐIỀU KIỆN XÉT VÀ CÔNG NHẬN TỐT NGHIỆP

ĐẠI HỌC

Điều kiện xét và công nhận tốt nghiệp
1. Sinh viên được công nhận và cấp bằng tốt nghiệp bậc đại học, loại hình đào tạo chính quy nếu hội đủ các điều kiện sau đây:
a) Tích lũy đủ học phần, số tín chỉ và hoàn thành các nội dung bắt buộc khác theo yêu cầu của CTĐT, đạt chuẩn đầu ra của CTĐT;
b) Điểm trung bình tích lũy của toàn khóa học đạt từ trung bình trở lên;
c) Tại thời điểm xét tốt nghiệp không bị truy cứu trách nhiệm hình sự hoặc không đang trong thời gian bị kỷ luật ở mức đình chỉ học tập.
2. Mỗi học kỳ chính, Hội đồng xét tốt nghiệp bậc đại học, loại hình đào tạo chính quy tiến hành xem xét và thông qua danh sách những sinh viên đủ điều kiện tốt nghiệp. Dựa trên đề nghị của Hội đồng xét tốt nghiệp, Hiệu trưởng ban hành quyết định công nhận tốt nghiệp.
3. Sinh viên tốt nghiệp 

Error at generate embeddings


ValueError: Length of values (769) does not match length of index (770)

In [11]:
len(leaf_texts)

602

In [11]:
results[1]

(                                                 text  \
 0   THÔNG TIN VỀ HỌC PHÍ, HỌC BỔNG DÀNH CHO SINH V...   
 1   HỆ THỐNG CHỈ DẪN\nUEH Wayfinding\nXEM NGAY\nKH...   
 2   .\n\nĐiều khiển thông minh và tự động hóa (thu...   
 3   .ueh.edu.vn/.\n\nTài khoản này sinh viên cũng ...   
 4   UEH SHUTTLE BUS\nTHUẬN TIỆN, AN TOÀN VÀ NHANH ...   
 5   .edu.vn\n\nPhòng A0.04, cơ sở A\n\n(028) 7306 ...   
 6   HỖ TRỢ THÔNG TIN – TƯ VẤN NGƯỜI HỌC\n\nPhòng C...   
 7   .ueh.edu.vn”, trong đó: http (Hyper Text Trans...   
 8   KHẢO SÁT CHẤT LƯỢNG HỌC PHẦN\n\nTrên tinh thần...   
 9   .\n\nNgoài ra, còn có những quy định riêng khá...   
 10  .\n\nThe opening of UEH English Zone is an exc...   
 11  .\n\nChương trình đào tạo song ngành bao gồm: ...   
 12  .\n7\tHoàn tất nghĩa vụ học phí.\tTừ lúc đăng ...   
 13  Trang chủ\nTư vấn trước nhập học\nĐại học chín...   
 14  .\n\nNgoài ra, còn có những quy định riêng khá...   
 15  . Sinh viên đăng ký học lại, học cải thiện điể...   
 16  CHƯƠNG TR

In [12]:
# Initialize all_texts with leaf_texts
all_texts = leaf_texts.copy()

# Iterate through the results to extract summaries from each level and add them to all_texts
for level in sorted(results.keys()):
    # Extract summaries from the current level's DataFrame
    summaries = results[level][1]["summaries"].tolist()
    # Extend all_texts with the summaries from the current level
    all_texts.extend(summaries)



In [13]:
es.add_texts(all_texts)

['47f38849-1954-48be-b704-da43d2597572',
 'c92bddf0-e0fa-4084-9e1e-b3c81ea886be',
 '0a7d6320-b765-477f-8292-fbc7d41c218c',
 '3d144d23-49aa-4626-b020-2aa3b4551823',
 'c204ed90-856b-4211-8dd7-08d87e92df1c',
 'b85102cc-02be-445e-a8ee-d2264d9e0f05',
 'f1217387-375b-46b9-9ccb-65742b57fc65',
 '8abd27c4-5109-4d33-bf6e-6644bcf961f9',
 '7b4ddbf4-3776-4bea-8d36-bdf1ca721efd',
 '3247568c-fdc3-47ac-8d22-83071f9adc97',
 '3be1129d-ed18-4be3-bd3a-258bd06affff',
 '51583add-e783-4a00-8de7-98f21286861a',
 'f936ff60-db0c-4f04-8949-81213ec52e98',
 '68c1a8b7-eb4d-4ad1-ad25-e319b444cfd5',
 '8f790270-b72b-4b35-88ce-7975120382bf',
 'bec563ca-1380-429d-9020-66f6240ca7a5',
 'c3742ed2-8d00-47cf-82df-2e84213a52f7',
 'ce8f4ef9-cb35-4e3c-a473-aba7f7930338',
 '843ecbca-61ba-4f4d-acde-aac07d76f263',
 'e925776e-4b38-4a89-b45c-b8b45f00d58b',
 '85711b1e-50a4-4a8d-9962-f5647de64c13',
 '6d67353c-6317-4bfd-b933-55913d440770',
 'b27f67c6-88bf-4818-8b5d-080b15744465',
 '7569b12b-7f1d-4b9d-90e5-4c459a91cc92',
 '2988d5dc-3503-

In [14]:
bm.add_texts(all_texts, metadatas={})

['21122dff-1f46-4787-bd63-713df8098a0e',
 'babbc467-c0ee-4375-b335-66c814e47e3d',
 '8aa1d03b-9bcd-4565-8345-f3a0c4a7a59c',
 '56b72a66-ae1a-40d6-8d70-54632a3e74ff',
 '8d9fc87c-eda0-4874-9c65-4449101b5ec3',
 'eb1499b6-5a27-4928-911f-c632276e233a',
 '3a75ef5a-8edd-48e1-890a-ae20db7e1b0d',
 'ae5df7e9-ced7-4b28-803e-00d143007198',
 'a3f683c5-bffa-4983-aa37-a5e8eb6bd54f',
 'c249eb1c-d550-443c-adf3-f6aee6506eb9',
 'a83d023a-6c7e-4862-8927-dd5cb07b98af',
 'a6d13540-d452-4887-b1e1-f5e2784e6596',
 '3ca9b98c-9967-4f95-a8d9-c5b4e7199613',
 '18850611-9bd9-4284-b6f6-c3aac3ff101e',
 '1c986f23-72e9-4334-bd40-a87e0b8e838e',
 '6400486c-4234-4bfb-914d-c9d7c2d66ff8',
 '877bb1c2-9c9f-404e-87d9-5fa4a13a1904',
 '59989e58-62ba-4e2e-a497-92cac7cd10de',
 'de8e116c-47de-4763-854e-c5e0df08997f',
 '0e3fd5d3-39ea-4aae-aa1d-bd7c584b400d',
 '5ac1fe16-8858-4d61-b578-fc6407b39d46',
 'deeba63c-c079-4bd5-af8b-84fd2b02926d',
 'f22724a4-5f00-46a5-b452-38c3ffe19810',
 'ecb5a0a7-a79f-4c30-bebb-4509c583dc0e',
 '0ad40243-1033-

In [5]:
from src.rag.hyde_rag import HydeRAG, HydeHybridSearchRAG
from src.utils.config_utils import get_gemini_hyde_config

# rag = HydeRAG(provider=provider, index="test-basic_test_tdt_dataset")
config = get_gemini_hyde_config()
config.vector_index = vec_idx
config.text_index = text_idx

# config.vector_index = "vec-sentence-propositon_medium_edu_tdt"
# config.text_index = 'text-sentence-propositon_medium_edu_tdt'

# rag = HydeRAG(provider=provider, index=config.vector_index)
rag = HydeHybridSearchRAG(provider=provider,config=config, k=4)



In [6]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
retriever = rag.ensemble_retriever
model = provider.get_simple_gemini_pro()
# Prompt
prompt = hub.pull("rlm/rag-prompt")


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Question
# rag_chain.invoke("How to define a RAG chain? Give me a specific code example.")

In [7]:
provider.config.enable_tracing("CODEBOT")

In [10]:
# Question

# ques = "Lệ phí tập sự nghề nghiệp,ôn và thi tốt nghiệp,khóa luận tính như thế nào"
ques = "Sinh viên muốn làm giấy xác nhận để đi xe buýt thì phải làm sao"
ques = "Sinh viên thi TOEIC/MOS do IIG tổ chức tại trường Đại học Tôn Đức Thắng nhưng CMND bị mờ, ép dẻo, ép lụa, ép lại, bong tróc mép để hở phần giấy bên trong thì làm thế nào"
ques = "Các lỗi vi phạm khi Sinh viên ở ký túc xá có liên quan hay ảnh hưởng gì với kết quả học tập rèn luyện gì trường học hay không"
answ = rag_chain.invoke(ques)

answ

'Tôi không tìm thấy thông tin về mối liên hệ giữa các lỗi vi phạm của sinh viên ở ký túc xá với kết quả học tập rèn luyện của họ trong tài liệu được cung cấp.'